In [1]:
  !pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2024.1.2-cp310-cp310-linux_x86_64.whl size=662246 sha256=ddc1a19e42908d77a5a5fe82bcdf872ff43b86325babd8e36a6d05319add6981
  Stored in directory: /root/.cache/pip/wheels/70/63/40/4bf006182f942d3516b71bb2ff3b57ccbdb8b2c0ee81882b6e
Successfully built pycuda


In [2]:
import pycuda.driver as drv
drv.init()
print ("%d device(s) found." % drv.Device.count())
for ordinal in range(drv.Device.count()):
  dev = drv.Device(ordinal)
  print ("Device #%d: %s" % (ordinal, dev.name()))
  print (" Compute Capability: %d.%d" % dev.compute_capability())
  print (" Total Memory: %s KB" % (dev.total_memory()//(1024)))
  print (" Maximum number of threads per block %s" % str(dev.MAX_THREADS_PER_BLOCK))
  print (dev.MULTIPROCESSOR_COUNT)

1 device(s) found.
Device #0: Tesla T4
 Compute Capability: 7.5
 Total Memory: 15464512 KB
 Maximum number of threads per block 1024
40


In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b4609bd1-0ecc-06b1-b454-6bd0bd6f4877)


In [3]:
from google.colab import drive
import pandas as pd
import numpy
drive.mount('/content/drive')
songs = pd.read_csv('/content/drive/My Drive/Colab Notebooks/yataros1.csv',header=None)

dataset_sec_rec=songs.copy()
dataset_sec_rec=dataset_sec_rec.transpose()

dataset_sec_rec.drop(0, axis=1, inplace = True)
dataset_sec_rec = dataset_sec_rec.iloc[1:]
dataset_sec_rec = dataset_sec_rec.reset_index(drop=True)
dataset_sec_rec = dataset_sec_rec.to_numpy()
dataset_sec_rec=dataset_sec_rec.astype(numpy.float32)

# songs=songs.transpose()
# print(songs.isnull().sum())

songs.drop(0, axis=1, inplace = True)
songs = songs.iloc[1:]
songs = songs.reset_index(drop=True)
dataset = songs.to_numpy()

# dataset=dataset[0:10,0:12]
dataset=dataset.astype(numpy.float32)
n=numpy.size(dataset,axis=0)
# print(numpy.size(dataset,axis=0),numpy.size(dataset,axis=1))
# print(dataset)
cents_graph_global=[]
# print(n)

Mounted at /content/drive


In [4]:
##PYCUDA

import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
import numpy
import math

mod = SourceModule("""
__global__ void edm(float *X_d,int N,int F,float *D_d)
{

int thisRow=threadIdx.y+blockIdx.y*blockDim.y;
int otherRow=threadIdx.x+blockIdx.x*blockDim.x;
float d;
int m;

if (thisRow< F && otherRow <F){
  d=0;
  for(int f = 0; f < N; f++) {
    int i=thisRow*N+f;
    int j=otherRow*N+f;
    d=d+pow((X_d[i]-X_d[j]),2);
  }
  m=thisRow*F+otherRow;
  D_d[m]=sqrt((float) d);
}
}

__global__ void KCentresGPU(float *D_d,int *K_d,int *L_d,int N, int K)
{
int i=threadIdx.x+blockIdx.x*blockDim.x;
float minimum=float('inf');
int pos=-1;
int kk,m;

if (i<N){
  for(int k = 0; k < K; k++) {
    kk=K_d[k];
    m=i*N+kk;

    if (D_d[m]<minimum){
      minimum=D_d[m];
      pos=k;
    }
  }
  L_d[i]=pos;
}

}

__global__ void findMaximaGPU(float *D_d,int *L_d,float *M_d,int N)
{
int i=threadIdx.x+blockIdx.x*blockDim.x;
float maximum=-1.0;

int m1,m2;

if (i<N){
  int cluster_label=L_d[i];
  for(int j = 0; j < N; j++) {
    if(cluster_label==L_d[j]){
      m1=i*N+j;
      if(D_d[m1]>maximum){
        maximum=D_d[m1];
      }
    }
  }
  m2=cluster_label*N+i;
  M_d[m2]=maximum;

}
}

__global__ void findMinimumOfMaximaGPU(int *t_d,float *M_d,int N,int K)
{
  int k=threadIdx.x+blockIdx.x*blockDim.x;
  float minimum=float('inf');
  int m3,pos;

  if(k<K){
    for(int j=0;j<N;j++){
      m3=k*N+j;
      if(M_d[m3]<minimum){
        minimum=M_d[m3];
        pos=j;
      }
    }
    t_d[k]=pos;
  }
}
""")


<ipython-input-4-486ab84a048e>:9: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu(26): warning #1422-D: multicharacter character literal (potential portability problem)
  float minimum=float('inf');
                      ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

kernel.cu(71): warning #1422-D: multicharacter character literal (potential portability problem)
    float minimum=float('inf');
                        ^


  mod = SourceModule("""


In [6]:
from timeit import default_timer
import time

def findFirstCentres(k_length,n):
  #L
  L=numpy.full((n),-1,dtype=numpy.int32)
  L_gpu=cuda.mem_alloc(L.nbytes)
  cuda.memcpy_htod(L_gpu,L)

  #t
  t=numpy.full((k_length),-1,dtype=numpy.int32)
  t_gpu=cuda.mem_alloc(t.nbytes)
  cuda.memcpy_htod(t_gpu,t)

  #M
  M=numpy.full((k_length,n),numpy.inf)
  M = M.astype(numpy.float32)
  M_gpu = cuda.mem_alloc(M.nbytes)
  cuda.memcpy_htod(M_gpu, M)


  return L_gpu,L,t_gpu,t,M_gpu,M

def main(dataset,k_length,n,nrep=1):
  elapsed_time=0

  L_gpu,L,t_gpu,t,M_gpu,M=findFirstCentres(k_length,n)

  func_EDM = mod.get_function("edm")
  func_KCentresGPU = mod.get_function("KCentresGPU")
  func_findMaximaGPU = mod.get_function("findMaximaGPU")
  func_findMinimumOfMaximaGPU = mod.get_function("findMinimumOfMaximaGPU")

  dataset = dataset.astype(numpy.float32)
  dataset_gpu = cuda.mem_alloc(dataset.nbytes)
  cuda.memcpy_htod(dataset_gpu, dataset)

  F=numpy.size(dataset, 1)
  F= numpy.int32(F)
  N= numpy.size(dataset,axis=0)
  N= numpy.int32(N)
  k_length= numpy.int32(k_length)

  mat_dis=numpy.empty([F, F])
  mat_dis = mat_dis.astype(numpy.float32)
  mat_dis_gpu = cuda.mem_alloc(mat_dis.nbytes)
  cuda.memcpy_htod(mat_dis_gpu, mat_dis)
  N_mat_dis= numpy.size(mat_dis,axis=0)
  N_mat_dis= numpy.int32(N_mat_dis)

  threads=dev.MAX_THREADS_PER_BLOCK
  threads1=math.sqrt(threads)
  threads1= int(threads1)

  gridx=int((N+threads1-1)/threads1)
  gridy=int((N+threads1-1)/threads1)
  inicio=default_timer()

  ##MEDIDA
  start_time=time.time()
  func_EDM(dataset_gpu,N,F,mat_dis_gpu,block=(threads1,threads1,1),grid=(gridx,gridy,1))

  end_time=time.time()
  print("Tiempo de Matriz de distancias: ",end_time - start_time)

  cuda.memcpy_dtoh(mat_dis, mat_dis_gpu)


  dimx1 = int(math.sqrt(threads));
  block1=(dimx1, 1,1);
  grid1=(int((n + block1[0] - 1) / block1[0]), 1,1);
  dopt=numpy.inf

  cents_graph_pyCuda=[]
  for i in range(nrep):


    K=numpy.array([3, 14, 35, 24 , 39], dtype=numpy.int32)
    K_gpu = cuda.mem_alloc(K.nbytes)
    cuda.memcpy_htod(K_gpu, K)

    start_time = time.time()

    while True:
      ##MEDIDA TIEMPO
      start_time=time.time()

      func_KCentresGPU(mat_dis_gpu,K_gpu,L_gpu,N_mat_dis,k_length,block=block1,grid=grid1)

      cuda.memcpy_dtoh(L, L_gpu)

      func_findMaximaGPU(mat_dis_gpu,L_gpu,M_gpu,N_mat_dis,block=block1,grid=grid1)

      func_findMinimumOfMaximaGPU(t_gpu,M_gpu,N_mat_dis,k_length,block=block1,grid=grid1)


      end_time=time.time()
      elapsed_time+= (end_time - start_time)


      cuda.memcpy_dtoh(t, t_gpu)
      # cuda.memcpy_dtoh(K, K_gpu)


      if(numpy.all(t==K)):
        dmin,labs=numpy.min(mat_dis[t,:],axis=0),numpy.argmin(mat_dis[t,:],axis=0)
        dmin=numpy.max(dmin)
        cents_graph_pyCuda.append(t.tolist())
        break
      K=t.copy()
      # K_gpu=t_gpu
      cuda.memcpy_dtod(K_gpu, t_gpu, K.nbytes)

    finish_time = time.time()
    elapsed_time+=(finish_time-start_time)

    #almacena los centros más optimos en jopt.
    if(dmin<=dopt):
        dopt=dmin
        labels=numpy.transpose(labs)
        jopt=t.tolist()


  #Determina los mejores centros sobre los N intentos.
  dm=numpy.zeros(k_length)
  for i in range(k_length):
      L=numpy.where(labels==i)[0]
      dm[i]=numpy.max(mat_dis[jopt[i],L])

  cents_graph_global.append(cents_graph_pyCuda)

  fin=default_timer()


  print("Tiempo de K-centres: ",elapsed_time/nrep)

  return labels,jopt,dm


k_length=5

n=numpy.size(dataset,axis=1)

labels,jopt,dm=main(dataset,k_length,n,25)

print("\n",dm,labels)
print("\nCENTROIDES :",jopt)





<ipython-input-6-9baa2b71d58a>:45: RuntimeWarning: overflow encountered in cast
  mat_dis = mat_dis.astype(numpy.float32)


Tiempo de Matriz de distancias:  7.486343383789062e-05
Tiempo de K-centres:  0.00038193702697753907

 [1.87371984e+08 9.36760559e+01 5.79447578e+04 1.35219893e+04
 8.05443848e+02] [2 3 3 3 3 1 4 1 1 4 1 1 1 1 1 1 1 0 0 0 0 2 0 1 1 1 1 1 1 1 1 4 1 3 1 4 1
 1 4 1]

CENTROIDES : [19, 24, 0, 2, 9]
